In [2]:
!pip install SciKit-Learn
!pip install joblib


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
#import dataset
from google.colab import files
import pandas as pd

uploaded = files.upload()
print(uploaded.keys())



Saving Kepler_Dataset_edited_SNR.csv to Kepler_Dataset_edited_SNR (4).csv
dict_keys(['Kepler_Dataset_edited_SNR (4).csv'])


In [28]:
df1 = pd.read_csv("Kepler_Dataset_edited_SNR (4).csv", sep=",", comment="#")


print("Kepler shape:", df1.shape)

print("\nKepler sample:")
display(df1.head())


Kepler shape: (9564, 49)

Kepler sample:


,kepid,kepoi_name,kepler_name,status,koi_pdisposition,score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,surface_gravity,koi_slogg_err1,koi_slogg_err2,stellar_radius,koi_srad_err1,koi_srad_err2,ra,dec,apparent_brightness
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [29]:
print("Kepler columns:")
print(df1.columns.tolist())

# Orbital_Period                    koi_period        pl_orbper
# Transit_Duration                  koi_duration      pl_trandurh
# Planet_Radius                     koi_prad          pl_rade
# Stellar_Radius                    koi_srad          st_rad
# Transit_Depth                     koi_depth         pl_trandep
# Stellar_Effective_Temperature     koi_steff         st_teff
# Planet_Equillibrium_Temperature   koi_teq           pl_eqt                  ADD THIS

Kepler columns:
['kepid', 'kepoi_name', 'kepler_name', 'status', 'koi_pdisposition', 'score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'orbital_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'transit_impact_parameter', 'koi_impact_err1', 'koi_impact_err2', 'transit_duration', 'koi_duration_err1', 'koi_duration_err2', 'transit_depth', 'koi_depth_err1', 'koi_depth_err2', 'planet_radius', 'koi_prad_err1', 'koi_prad_err2', 'planet_equillibrium_temperature', 'koi_teq_err1', 'koi_teq_err2', 'stellar_insolation', 'koi_insol_err1', 'koi_insol_err2', 'signal_noise_ratio', 'koi_tce_plnt_num', 'koi_tce_delivname', 'stellar_effective_temperature', 'koi_steff_err1', 'koi_steff_err2', 'surface_gravity', 'koi_slogg_err1', 'koi_slogg_err2', 'stellar_radius', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec', 'apparent_brightness']


In [30]:
# Check common columns (removed "ra" and "dec")
df1 = df1.drop(columns=["ra", "dec"], errors="ignore")

combined=pd.concat([df1])

common_cols = set(df1.columns)
print("Common columns after dropping ra and dec:")
print(sorted(common_cols))


Common columns after dropping ra and dec:
['apparent_brightness', 'kepid', 'kepler_name', 'kepoi_name', 'koi_depth_err1', 'koi_depth_err2', 'koi_duration_err1', 'koi_duration_err2', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_impact_err1', 'koi_impact_err2', 'koi_insol_err1', 'koi_insol_err2', 'koi_pdisposition', 'koi_period_err1', 'koi_period_err2', 'koi_prad_err1', 'koi_prad_err2', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad_err1', 'koi_srad_err2', 'koi_steff_err1', 'koi_steff_err2', 'koi_tce_delivname', 'koi_tce_plnt_num', 'koi_teq_err1', 'koi_teq_err2', 'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'orbital_period', 'planet_equillibrium_temperature', 'planet_radius', 'score', 'signal_noise_ratio', 'status', 'stellar_effective_temperature', 'stellar_insolation', 'stellar_radius', 'surface_gravity', 'transit_depth', 'transit_duration', 'transit_impact_parameter']


In [68]:
#Convert Data into a managable format for SciKit-Learn

Xvals= ['orbital_period', 'planet_radius', 'stellar_effective_temperature'
, 'stellar_radius', 'transit_depth', 'transit_duration', 'apparent_brightness', 'surface_gravity', 'stellar_insolation',
'transit_impact_parameter', 'planet_equillibrium_temperature'
]
Yvals= 'status'


df1["radius_ratio"] = df1["planet_radius"] / (df1["stellar_radius"] + 1e-6)
df1["depth_duration_ratio"] = df1["transit_depth"] / (df1["transit_duration"] + 1e-6)
df1["snr_radius_product"] = df1["signal_noise_ratio"] * df1["planet_radius"]

Xvals.extend(["radius_ratio", "depth_duration_ratio", "snr_radius_product"])


X = df1[Xvals]
Y = df1[Yvals]


X=np.array(X)
Y=np.array(Y)

#can't use strings to train the model, turn status into numbers
for i in range (len(Y)):
  if Y[i] in ['CANDIDATE', 'ACANDIDATE', 'APC']:
    Y[i]=0
  elif Y[i]=='CONFIRMED':
    Y[i]=1
  elif Y[i] in ['FALSE POSITIVE', 'FA']:
    Y[i]=2

print(X, Y)

[[9.48803557e+00 2.26000000e+00 5.45500000e+03 ... 2.43796932e+00
  2.08283953e+02 8.09080000e+01]
 [5.44183827e+01 2.83000000e+00 5.45500000e+03 ... 3.05285539e+00
  1.94142402e+02 7.30140000e+01]
 [1.98991399e+01 1.46000000e+01 5.85300000e+03 ... 1.68202571e+01
  6.05992253e+03 1.11398000e+03]
 ...
 [6.81401611e-01 1.07000000e+00 6.17300000e+03 ... 1.02785684e+00
  1.20231075e+02 1.31610000e+01]
 [3.33486169e+02 1.93000000e+01 4.98900000e+03 ... 2.46676860e+00
  1.99749859e+02 2.70200000e+02]
 [4.85603482e+00 1.05000000e+00 6.46900000e+03 ... 8.80133378e-01
  2.49187703e+01 8.61000000e+00]] [1 1 0 ... 2 0 2]


In [81]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier



# Use a stratified shuffle split
Split= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=69)
for train_index, test_index in Split.split(X, Y):
  X_train= X[train_index]
  X_test= X[test_index]
  Y_train= Y[train_index]
  Y_test= Y[test_index]


Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)


model= RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    bootstrap=True,
    class_weight='balanced_subsample',
    random_state=69,
    n_jobs=-1)
model.fit(X_train, Y_train)
Yprediction=model.predict(X_test)
print(Yprediction, Y_test)
print(100* accuracy_score(Y_test, Yprediction), "%")


[1 2 2 ... 2 0 2] [0 2 2 ... 2 1 0]
76.52901202300052 %


In [82]:
#Save the model
import joblib
joblib.dump(model, 'ExoPlanet_Classifier.joblib')
model=joblib.load('ExoPlanet_Classifier.joblib')

In [83]:
from joblib import dump

# Save the trained model
dump(model, "kepler_model.joblib")

print("✅ Model saved as kepler_model.joblib")

from google.colab import files
files.download("kepler_model.joblib")



✅ Model saved as kepler_model.joblib


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>